# Pattern Generation Notebook
Contents:
- [Prerequisites](#Prerequisites)
- [Pattern Generation](#pattern-generation)
    - [Settings](#settings)
    - [Static Generation](#static-generation)
    - [REED Patterns](#reed-patterns)
    - [Dynamic Generation](#dynamic-generation)
- [Generators](#generators)
    - [Static Generator](#static-generator)
    - [Dynamic Generator](#dynamic-generator)
- [Simulated Annealing](#simulated-annealing)

## Prerequisites
To run the scripts in this notebook we require some modules.
To install the modules run the code below.

In [ ]:
# YOU MUST BE RUNNING PYTHON 3.7.X
# HIGHER VERSIONS DO. NOT. WORK.
from platform import python_version
from io import UnsupportedOperation
if python_version()[0:3] != '3.7':
    raise Exception("You must be running Python 3.7.X")

# System libraries
# Json may appear to not be used, but it's used for commented-out conversion code.
import json, os, sys, string, random, subprocess, time, shutil, datetime
from collections.abc import Callable, Mapping
from typing import Tuple
print("System libraries imported.")

# Import external modules
try:
    import gifutils # This is a local library with dependencies, so it is in this list too.
    # If you run into an issue with it, please make sure the dependencies are in requirements.txt, or resolve it manually.
    import numpy as np
    import scipy.signal
    import wget
    import psutil
except ImportError:
    print("Required libraries missing, installing them...")
    subprocess.call(['pip3', 'install', '-r', 'requirements.txt', '--no-warn-script-location'])
    import gifutils
    import numpy as np
    import scipy.signal
    import wget
    import psutil
print("All libraries imported.")

# Import visil package
try:
    from visil.model.visil import ViSiL
    from visil.datasets import load_video
except ImportError:
    print("Missing visil package...")

    import tensorflow as tf
    if tf.__version__ != '1.15.1':
        print("Cloning original (non-compat) package...")
        subprocess.call(['git', 'clone', 'https://github.com/MKLab-ITI/visil'])
    else:
        print("Cloning compat package...")
        subprocess.call(['git', 'clone', 'https://github.com/teamhart-nl/visil'])
    print("Installing dependencies...")
    subprocess.call(['pip3', 'install', '-r', 'visil/requirements.txt', '--no-warn-script-location'])
    from visil.model.visil import ViSiL
    from visil.datasets import load_video
print("Visil package loaded.")

# Load model
try:
    if not os.path.exists('ckpt/resnet'):
        raise Exception("L")
    print("Loading model. Tensorflow deprecation warnings are expected but safe to ignore.")
    model = ViSiL('ckpt/resnet/')
except Exception:
    print("Required model missing, installing it...")
    from zipfile import ZipFile
    if not os.path.exists('ckpt.zip'):
        print("Downloading model from http://ndd.iti.gr/visil/ckpt.zip, this may take a while...")
        wget.download('http://ndd.iti.gr/visil/ckpt.zip')
    print("Unzipping model...")
    ZipFile('ckpt.zip', 'r').extractall()
    print("Loading model...")
    model = ViSiL('ckpt/resnet/')
print("Model loaded.")



In [ ]:
# Model Verification
# Let's try loading the 2 example videos and calculating the score
# This is optional, but it's a good way to check if the model is working
v1 = load_video('visil/examples/video1.gif')
v2 = load_video('visil/examples/video2.gif')
v1features = model.extract_features(v1, batch_sz=32)
v2features = model.extract_features(v2, batch_sz=32)
similarity = model.calculate_video_similarity(v1features, v2features)
similarity = round(similarity, 2)
"Successfully setup model!" if similarity > 0.3 and similarity < 0.8 else "Model setup failed!"


## Pattern Generation
We generate patterns using one of two methods:
- [Static Generation](#static-generation)
- [Dynamic Generation](#dynamic-generation)

Static generation generates a sequence of frames where the same actuators are used in all frames.
Dynamic generation, on the other hand, generates a sequence of frames with different actuators, that give a shape or motion.

In [ ]:
# Settings
FPS = 10
GRID_WIDTH = 4
GRID_HEIGHT = 6
MIN_ACTUATORS = 1
MAX_ACTUATORS = 8
AMPLITUDES = [100, 255]
FREQUENCIES = [69]
MODULATIONS = [8]
WAVEFORMS = ["sin", "sawtooth", "block", "hanning", "constant"]
CHANNELS = {
    "red": "amplitude",
    "green": "amplitude",
    "blue": "amplitude"
}

### Static Generation
Generates a sequence of frames where the same actuators are used in all frames.

*Description goes here*

In [ ]:
# type of E: 'set' which is a subset of (4,6), e in E, e[0] is '4', e[1] is '6'
# type of D: duration in of the pattern (integer in ms)
# type of G_f: frequency of wave type (integer)
# type of W: string, denoting the wave type

POSSIBLE_WAVEFORMS = ['sin', 'sawtooth', 'block', 'hanning', 'constant']


def staticPattern(E, a, f, d, g_f, w) -> list:
    """
    #TODO: add drawing somewhere

    We say 'extra' is a parameter either a or f.
    We say 'x' is from top view of the grid the x-coordinate
    We say 'y' is from top view of the grid the y-coordinate

    :param E: (array) subset of whole grid. E = E[e_1,e_2,...]. For each e in E: x=e[0], y=e[1]
    :param a: (int: mm) one of all amplitudes. a = a in A
    :param f: (int: Hz) one of all frequencies. f = f in F
    :param d: (int: ms) one of all durations. d = d in D ()
    :param g_f: (int) one of all group frequencies. g_f = g_f in G_f. Also known as 'modulation'
    :param w: (str) one of all wave types. w = w in W
    :return: array: (array) an array representing the pattern in video form. array = array[time][x][y][extra].
             note that array[][][][0] gives the frequency, and array[][][][1] gives the amplitude.
             """

    # initial values
    max_amp = 250
    delta_t = 10
    discretization = d // delta_t
    array = np.zeros(shape=(discretization,4,6,2)) #maybe useful to make this a custom class

    if w == 'constant':
        for timestep in range(discretization):
            for e in E:
                array[timestep][e[0]-1][e[1]-1][0] = f
                array[timestep][e[0]-1][e[1]-1][1] = a

    elif w == 'sin':
        # Note that we assume a sine wave y = A sin(B(x-phi))+D

        # Computed from input
        B = 2*np.pi*g_f
        A = (max_amp - 0.5*max_amp)/2   # assuming a sin wave with 0.5*max_amp amplitude
        D = max_amp - A                 # wave between max_amp and max_amp/2

        # For creating wave
        start = 0
        stop = d
        x = np.linspace(start, stop, discretization)

        #create amplitude list
        phi = 0
        amplitude_list = []
        for i in range(len(x)):
            amplitude_list.append((int) (A*np.sin(B*(x[i] - phi)) + D))

        for timestep in range(discretization):
            for e in E:
                array[timestep][e[0]-1][e[1]-1][0] = f
                array[timestep][e[0]-1][e[1]-1][1] = amplitude_list[timestep]

    elif w == 'sawtooth':
        # TODO: add explanation of theoretic function from signal.sawtooth

        # Computed from input
        B = 2*np.pi*g_f

        # for creating a wave
        start = 0
        stop = d
        x = np.linspace(start, stop, discretization)

        # Create amplitude list
        phi = 0
        amplitude_list = []
        for i in range(len(x)):
            amplitude_list.append((int) ((max_amp*scipy.signal.sawtooth(B * (x[i] - phi)) + max_amp)/2))

        for timestep in range(discretization):
            for e in E:
                array[timestep][e[0]-1][e[1]-1][0] = f
                array[timestep][e[0]-1][e[1]-1][1] = amplitude_list[timestep]

    elif w == 'hanning':
        # Note that we assume a hanning window y = 0.5(1 - cos(2pin/N)), 0 <= n <= N  # noqa: E501
        # This function is built into numpy as hanning(discretization_rate).

        #create amplitude list
        amplitude_list = max_amp * np.hanning(discretization) #hanning window

        for timestep in range(discretization):
            for e in E:
                array[timestep][e[0]-1][e[1]-1][0] = f
                array[timestep][e[0]-1][e[1]-1][1] = amplitude_list[timestep]

    elif w == 'block':
        # Note that we assume a block function y = 1 if 0 <= x < period, -1 if period < x < 2*period

        # can be calculated from input
        period = 1 / g_f

        # for creating wave
        start = 0
        stop = d
        x = np.linspace(start, stop, discretization)

        #create amplitude list
        phi = 0
        amplitude_list = []
        for i in range(len(x)):
            if x[i] % 2*period < period: # if 0 <= x < period
                sign = 1
            else: # period <= x < 2*period
                sign = -1
            amplitude_list.append((int) (a * sign))
        for timestep in range(discretization):
            for e in E:
                array[timestep][e[0]-1][e[1]-1][0] = f
                array[timestep][e[0]-1][e[1]-1][1] = amplitude_list[timestep]
    else:
        print("error: wave type unknown")
        sys.exit(1)

    return array

"""
:param name: (str) name of the pattern
:param grid_width: (int) width of the grid
:param grid_height: (int) height of the grid
:param actuators_no: (tuple) -> (lower, upper) bound on amounts of actuators enabled. 
:param amplitudes: (list) list of amplitudes
:param frequencies: (list) list of frequencies
:param modulations: (list) list of modulation frequencies
:param durations: (list) list of durations
:param waveforms: (list) waveform options, pick one or more from from ['sin', 'sawtooth', 'block', 'hanning', 'constant']
:return: tuple of (name: str, pattern: list). Pattern representing the pattern in video form. pattern = pattern[time][x][y][extra].
         note that pattern[][][][0] gives the frequency, and pattern[][][][1] gives the amplitude.
"""
def generateRandomPattern(
    name: str,
    grid_width: int = 4,
    grid_height: int = 6,
    actuators_no: tuple = (1, 8),
    amplitudes: list = [100, 255],
    frequencies: list = [69],
    modulations: list = [8],
    durations: list = [92, 392],
    waveforms: list = POSSIBLE_WAVEFORMS) -> tuple:

    actuators = random.randint(actuators_no[0], actuators_no[1])
    amplitude = random.choice(amplitudes)
    frequency = random.choice(frequencies)
    modulation = random.choice(modulations)
    duration = random.choice(durations)
    waveform = random.choice(waveforms)
    waveform = random.choice(POSSIBLE_WAVEFORMS) if not waveform in POSSIBLE_WAVEFORMS else waveform

    enabled_actuators = [(random.randint(1,grid_width), random.randint(1,grid_height)) for _ in range (actuators)]

    return (name, staticPattern(enabled_actuators, amplitude, frequency, duration, modulation, waveform))

"""
:param name: (str) name of the pattern
:param grid: np.ndarray representing the pattern in video form. gird = grid[time][x][y][extra].
         note that grid[][][][0] gives the frequency, and grid[][][][1] gives the amplitude.
:param out_directory: (str) directory to save the patterns to
:param grid_width: (int) width of the grid
:param grid_height: (int) height of the grid
:param channel_red: (str) amplitude or frequency
:param channel_green: (str) amplitude or frequency
:param channel_blue: (str) amplitude or frequency
"""
def generateGIF(
    name: string,
    grid: np.ndarray,
    out_directory: str = 'gifs',
    grid_width: int = 4,
    grid_height: int = 6,
    channel_red: str = "amplitude",
    channel_green: str = "amplitude",
    channel_blue: str = "amplitude") -> None:

    grids = [[[[255, 255, 255] for _ in range(grid_width)] for _ in range(grid_height)] for _ in range(len(grid))]

    for index, _ in np.ndenumerate(grid):
        # index:
        # (i, j, k, l) where:
        frame = grid[index[0]]

        for column_id, column in enumerate(frame):
            for row_id, row in enumerate(column):
                if channel_red == "frequency" or channel_green == "frequency" or channel_blue == "frequency":
                    raise UnsupportedOperation("frequency channel not supported yet")
                amp = frame[column_id][row_id][1]
                red = 255 - amp
                green = 255 - amp
                blue = 255 - amp
                grids[index[0]][row_id][column_id] = [red, green, blue]

    gifutils.save_frames_as_gif(gifutils.frames_from_lists(grids), out_directory, name)

### REED patterns
The set of patterns from the REED paper. These use the static method.

In [ ]:
from collections import namedtuple

sp = namedtuple('static_pattern', 'phoneme total_time modulation fraction phi dis coord_list pho_freq')

reed_data = [
    sp('B',  92, 30, 0.5, 0, 6, [[1,5],[1,6],[2,5],[2,6]], 300),
    sp('M', 392, 8, 0.5, 0, 12,  [[1,5],[1,6],[2,5],[2,6]], 60),
    sp('J', 392, 8, 0.5, 0, 12, [[1,1],[1,6],[2,1],[2,6]], 300),
    sp('D',  92, 30, 0.5, 0, 6, [[3,3],[3,4],[4,3],[4,4]], 300),
    sp('G',  92, 30, 0.5, 0, 6, [[1,1],[1,2],[2,1],[2,2]], 300),
    sp('V', 392, 8, 0.5, 0, 12, [[1,6],[2,6],[3,6],[4,6]], 300),
    sp('DH',392, 8, 0.5, 0, 12,[[1,3],[1,4],[2,3],[2,4]], 300),
    sp('Z', 392, 8, 0.5, 0, 12, [[1,1],[2,1],[3,1],[4,1]], 300),
    sp('ZH',392, 8, 0.5, 0, 12,[[3,1],[3,2],[4,1],[4,2]], 60),
    sp('N', 392, 8, 0.5, 0, 12, [[3,3],[3,4],[4,3],[4,4]], 60),
    sp('NG',392, 8, 0.5, 0, 12,[[1,1],[1,2],[2,1],[2,2]], 60),
    sp('W', 392, 8, 0.5, 0, 12, [[1,3],[1,4],[1,5],[1,6],[3,3],[3,4],[3,5],[3,6]], 60),
    sp('L', 392, 30, 0.5, 0, 12,[[3,5],[3,6],[4,5],[4,6]], 300),
    sp('R', 392, 30, 0.5, 0, 12,[[3,1],[3,2],[4,1],[4,2]], 300),
]

In [ ]:
# Deprecated Type definitions
E = []
a = -1
d = -1
f = -1
g_f = -1
w = ''

### Dynamic Generation

In [ ]:
# # type of E_j: 'set' which is a subset of (4,6), e in E, e[0] is '4', e[1] is '6'
# P_j is for j=3: [P_1, P_2, P_3]
# P_j is a list of 2 rows, where
# 	P_j[0] contains number dt
# 	P_j[1] consists of a list of [E_0, E_1, ..., E_k] such that
# 	each exciter e in E_i starts at time i * dt

# D_j is a list of durations, where D_j[i] is the amount of
# 	time the exciters in P_j[1][i] is active
# A_j is a list of amplitudes where A_j[i] is the amplitude
# 	of the group wave for the exciters in P_j[1][i]
# F_j
# G_F_j
# W_j

# P[j][E[j]][0] is a set of e's, a subset of (4,6), with dT on last index

def dynamicPattern(P, D, A, F, G_F, W):
	"""
	#TODO: add drawing somewhere

	We say 'extra' is a parameter either a or f.
	We say 'x' is from top view of the grid the x-coordinate
	We say 'y' is from top view of the grid the y-coordinate

	We say 'dt[i]' is t_start[i] - t_start[i-1] (see drawing)

	:param P: (array) P = P[P_1,P_2,...]. P_j=[P_j[0],P_j[1]], where P_j[0] = dt,
		      P_j[1] = [E_0,E_1,...], where e in E_i starts at time i*dt.
	:param D: (array) list of durations. D_j = [D_j[0],D_j[1],...],
			  where D_j[i] is the amount of time the exciters in P_j[1][i] is active.
	:param A: (array) list of amplitudes. A_j = [A_j[0],A_j[1],...],
			  where A_j[i] is the amplitude of the exciters in P_j[1][i].
	:param F: (array) list of frequencies. F_j = [F_j[0],F_j[1],...],
			  where F_j[i] is the frequency of the exciters in P_j[1][i].
	:param G_F: (array) list of group frequencies. G_F_j = [G_F_j[0],G_F_j[1],...],
			  where G_F_j[i] is the group frequency of the exciters in P_j[1][i].
	:param W: (array) list of wave types. W_j = [W_j[0],W_j[1],...],
			  where W_j[i] is the wave type of the exciters in P_j[1][i].
	:return: array: (array) an array representing the pattern in video form. array = array[time][x][y][extra].
			 note that array[][][][0] gives the frequency, and array[][][][1] gives the amplitude.
	"""

	discretization = sum([sum(D[i]) for i in range(len(D))]) // 10
	array = np.zeros(size=(discretization,4,6,2)) #maybe useful to make this a custom class


	# for j in range(len(P)):
	# 	for i in range(len(P)):
	# 		if W[j][i] == 'constant':
	# 			for timestep in range(D[][]):
	# 				for e in [P[j][1][i]]:
	# 					array[timestep][P[j][1][i]][0] =
	# 					array[timestep][P[j][1][i]][0] =

# Example input, |j|=2
dt_example = [0 for _ in range(2)]

e_111 = [2, 4]
e_112 = [2, 3]
E_11 = [e_111, e_112]

e_121 = [1, 2]
e_122 = [2, 2]
E_12 = [e_121, e_122]

E_example = [E_11, E_12]
P_1 = [dt_example, E_example] #E['dt',[E_11,E_12]]
P_2 = P_1
P = [P_1,P_2]  # [[[dt_example],[[E_11],[E_12]]], [[dt_example],[[E_21],[E_22]]]]
print(P)

D_1 = [400, 400]
D_2 = [200, 200]
D = [D_1, D_2]
# print(sum([sum(D[i]) for i in range(len(D))]))

A_1 = [250, 250]
A_2 = [250, 250]
A = [A_1, A_2]

F_1 = [300, 300]
F_2 = [60, 60]
F = [F_1, F_2]

G_F_1 = [30, 30]
G_F_2 = [30, 30]
G_F = [G_F_1, G_F_2]

W_1 = ['constant', 'constant']
W_2 = ['sawtooth', 'sawtooth']
W = [W_1, W_2]

## Generators
### Static Generator
Generate static patterns

In [ ]:
ACTUATORS_MIN = 4
ACTUATORS_MAX = 8
N_PATTERNS = 3000 # 1000 take about 1 minute to generate
GIF_DIRECTORY = 'gifs/'
for i in range(N_PATTERNS):
    p_name = 'p_' + str(i)
    if os.path.exists(GIF_DIRECTORY + p_name + ".gif"):
        continue
    generateGIF(p_name, generateRandomPattern(p_name, actuators_no=(ACTUATORS_MIN, ACTUATORS_MAX))[1])

## Simulated Annealing
We have 44 phonemes that require a pattern. These patterns need to be distinguishable from eachother, to the highest degree possible.  
Below is a model which can compare two patterns, pairwise only. At the bottom an overview of the methods of pattern collection is given.

#### Pairwise Model
https://github.com/MKLab-ITI/visil provides a pretrained model for video distinguishability (referred to as ViSiL).
We shall use this model as a baseline for pairwise pattern distinguishability.

#### Complete Patterns Model
The above model gives us the possibility to calculate the total score of a pair, but does not provide a method for quickly calculating the score of a collection of patterns.
There is of course the brute force method, generating a new set of patterns and getting the score for them, improving over time, but this is not practical due to runtime.
So we will be using Simulated Annealing (SA) to find the best set of patterns.

#### Simulated Annealing (SA)
Simulated Annealing is a method where the score of a collection of patterns is improved by starting with a random collection of patterns, and then iteratively changing the collection of patterns by (sub)randomly changing one of the patterns.

In [ ]:
__PATTERN_CACHE__ = {}
def make_or_get_cached_pattern(file: str):
    filehash = hash(file)
    if filehash not in __PATTERN_CACHE__:
        __PATTERN_CACHE__[filehash] = Pattern(file)
    return __PATTERN_CACHE__[filehash]

class Pattern:

    def __init__(self, file: str):
        self.file = file
        self.features = model.extract_features(load_video(GIF_DIRECTORY + "/" + file), batch_sz=32)

    def get_features(self):
        return self.features
    
    def __str__(self):
        return self.file.replace(".gif", "").replace("p_", "")
        
    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.file == other.file

    def __hash__(self):
        return hash(self.file)

Pattern score calculation

In [ ]:
def get_pair_score_function(x: Pattern, y: Pattern):
    if x == y:
        return 1
    return model.calculate_video_similarity(x.get_features(), y.get_features())

Pattern retrieval

In [ ]:
GIF_FILES = [f for f in os.listdir(GIF_DIRECTORY) if f.endswith('.gif')] # Note: any newly generated patterns are not in this list
if len(GIF_FILES) == 0:
    print('No GIFs found!')

In [ ]:
# Get a pattern that is not in existing_set
def get_new_pattern(existing_set: list = []) -> Pattern:
    tries = 0
    pattern: Pattern = None
    while tries < 1000:
        pattern = make_or_get_cached_pattern(random.choice(GIF_FILES))
        if pattern in existing_set:
            tries += 1
            continue
        return pattern

"estimated " + str(sys.getsizeof(get_new_pattern())) + " bytes per pattern"

Settings

In [ ]:
SET_SIZE = 44 # Amount of patterns in the set
COOLING_RATE = 0.99995 # Factor per iteration
INITIAL_TEMP = 0.005 # Initial temperature. Temp is chance to accept worse solution.
MIN_TEMP = 0.0000005 # Minimum temperature.
SCORE_PICKING = np.median # Algorithm to pick set score. Should take a list of float (float32) and return a float (float32).
UPDATE_TIME_MS = 10_000 # Miliseconds for the next update
BEST_SET = None # Best set found so far, used below \/
INITIAL_SET: list = None if BEST_SET == None else [make_or_get_cached_pattern("p_" + str(x) + ".gif") for x in BEST_SET] # Initial set of patterns. If None, will be generated randomly.
ITERATIONS = round(np.log(MIN_TEMP / INITIAL_TEMP) / np.log(COOLING_RATE)) # Number of iterations until lower bound is reached.
str(ITERATIONS) + " iterations!"

Util Functions

In [ ]:
# Some debug util methods
def round_to_sign_digits(x: float, digits: int = 2) -> str:
    return '{:g}'.format(float('{:.{p}g}'.format(x, p=digits)))

__SCORE_CACHE__: Mapping = {} # Do not modify this variable elsewhere!
def get_pattern_pair_score(x: Pattern, y: Pattern) -> np.float32:
    if (x.file, y.file) in __SCORE_CACHE__:
        return __SCORE_CACHE__[(x.file, y.file)]
    else:
        score = 1 - abs(get_pair_score_function(x, y))
        __SCORE_CACHE__[(x.file, y.file)] = score
        __SCORE_CACHE__[(y.file, x.file)] = score
        return score

# Debug the simulated annealing algorithm in a one-liner
__DEBUGCACHE__ = {'time': time.time(), 'iterations': -1}
def debug_sa(iteration: int, total_iterations: int, temperature: float, min_temp: float, current_score: float, current_set: list, additional_info: str = ""):
    if time.time() - __DEBUGCACHE__['time'] < UPDATE_TIME_MS / 1000 and iteration != 0:
        return

    time_since_update = (time.time() - __DEBUGCACHE__['time']) * 1000
    time_per_iteration = time_since_update / (iteration - __DEBUGCACHE__['iterations']) if iteration - __DEBUGCACHE__['iterations'] > 0 else 0
    time_ETA = (total_iterations - iteration) * time_per_iteration / 1000
    pattern_cache_filled = len(__PATTERN_CACHE__) / len(GIF_FILES) * 100
    score_cache_filled = len(__SCORE_CACHE__) / (len(GIF_FILES) ** 2) * 50

    message = '{}I {}/{} ({}%)\t{}ms/I, ETA {}s\tT={}% > {}%\tPC={}% / SC={}%\tS={}'.format(
        additional_info + " | " if additional_info != "" else "",
        iteration,
        total_iterations,
        round_to_sign_digits(iteration / total_iterations * 100, 3),
        round_to_sign_digits(time_per_iteration),
        round_to_sign_digits(time_ETA),
        round_to_sign_digits(temperature * 100),
        round_to_sign_digits(min_temp * 100),
        round_to_sign_digits(pattern_cache_filled, 4),
        round_to_sign_digits(score_cache_filled, 4),
        round_to_sign_digits(current_score, 4)
    )
    print(message)
    # write to gen.log
    with open('gen.log', 'a') as f:
        f.write(message + "\tC=" + str(current_set) + '\n')

    __DEBUGCACHE__['time'] = time.time()
    __DEBUGCACHE__['iterations'] = iteration

debug_sa(0, 1, 0, 0, False, [])

def get_set_score(element_list: list, score_picking: Callable) -> np.float32:
    scores = []
    for i, x in enumerate(element_list):
        for j, y in enumerate(element_list):
            if i == j:
                continue
            scores.append(get_pattern_pair_score(x, y))
            
    return score_picking(scores)

In [ ]:
def SA(element_set: list, score_picking: Callable = SCORE_PICKING, update_time: int = UPDATE_TIME_MS, temperature: float = INITIAL_TEMP,
       cooling_rate: float = COOLING_RATE, iterations: int = ITERATIONS, temp_lower_bound: float = MIN_TEMP) -> Tuple[Pattern, np.float32]:
    
    # Print SA details
    print("Starting {} iterations of Simulated Annealing on set of size {}, initial temp {}, temp lower bound {}, and a cooling rate of {}.\nUpdates every {} seconds, after an iteration finishes."
         .format(iterations, len(element_set), temperature, temp_lower_bound, cooling_rate, round_to_sign_digits(update_time / 1000, 3)))
    
    # Get the current score of the set
    print("Generating initial score. This may take a while...")
    current_score = get_set_score(element_set, score_picking)

    # 0th debug
    debug_sa(0, iterations, temperature, temp_lower_bound, current_score, element_set, "Initial")

    # Run through iterations
    for i in range(1, iterations + 1):

        # Create a copy of the set
        new_element_set = element_set.copy()

        # Replace a pattern in the set with a new one
        new_element_set[random.randint(0, len(new_element_set) - 1)] = get_new_pattern(new_element_set)
        
        # Get the score of the new set
        new_score = get_set_score(new_element_set, score_picking)

        # If the new score is better, keep the new set
        if new_score > current_score:
            element_set = new_element_set
            current_score = new_score

        # Else accept the new set with a probability
        else:
            if np.random.random() < np.exp((new_score - current_score) / temperature):
                element_set = new_element_set
                current_score = new_score

        # Cool the temperature
        temperature *= cooling_rate
        
        # Stop if the temperature is lower than the lower bound
        if temperature <= temp_lower_bound:
            debug_sa(i, iterations, temperature, temp_lower_bound, current_score, element_set, "Temperature lower bound reached")
            break

        # Print
        debug_sa(i, iterations, temperature, temp_lower_bound, current_score, element_set)
    
    # Print summary
    debug_sa(i, iterations, temperature, temp_lower_bound, current_score, element_set, "Final")
    
            
    # Return the new pattern set and its score
    return element_set, current_score

#### Pattern Caching

In [ ]:
PRECACHE_PATTERNS = True # It takes about 600MB of ram per 1000 patterns. It takes about 2 minutes per 1000 patterns to precache.
if PRECACHE_PATTERNS:
    start_ = time.time()
    start = time.time()
    memory = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2) # memory in MB
    for i, f in enumerate(GIF_FILES):
        if i % 100 == 0:
            print("Cached {}/{} patterns (took {}s, ETA {}s)".format(i, len(GIF_FILES), round_to_sign_digits(time.time() - start, 4), round_to_sign_digits((time.time() - start) / 100 * (len(GIF_FILES) - i), 4)))
            start = time.time()
        make_or_get_cached_pattern(f)
    print("Cached {} patterns in {}s, using approximately {}MB of memory".format(len(GIF_FILES), round_to_sign_digits(time.time() - start_, 4), round_to_sign_digits(psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2) - memory, 3)))

#### Code example
Below an example of how to generate patterns using SA. Uses many of the constants above. Also copies the best patterns to a folder.

In [ ]:
patternset = INITIAL_SET
if patternset is None:
    patternset = []
if not len(patternset) == SET_SIZE:
    print("Generating initial patterns...")
    while len(patternset) < SET_SIZE:
        patternset.append(get_new_pattern(patternset))
    while len(patternset) > SET_SIZE:
        print("Removing pattern due to oversizing...")
        patternset.pop()

start = time.time()

print("Running Simulated Annealing...")
patternset, score = SA(patternset)

print("Ran for {}s. Best retrieved set score is {}, with set: {}.".format(round_to_sign_digits(time.time() - start), round_to_sign_digits(score, 5), patternset))

In [ ]:
if not os.path.isdir("BEST"):
    os.mkdir("BEST")

for i in patternset:
    # Gif file name
    gif_file_name = "p_" + str(i) + ".gif"

    # Copy the gif file to a directory called "BEST" and subdirectory with today's date
    # Add the number to the end of the subdirectory name if it already exists.
    additional_number = 0
    datename = datetime.datetime.today().strftime("%Y-%m-%d")
    while os.path.isdir(os.path.join("BEST", datename)):
        additional_number += 1
        datename = datetime.datetime.today().strftime("%Y-%m-%d") + "_" + str(additional_number)

    os.mkdir(os.path.join("BEST", datename))
    shutil.copy(os.path.join(GIF_DIRECTORY, gif_file_name), os.path.join("BEST", datename, gif_file_name))